In [1]:
import os
import ast
import cirq
import numpy as np

In [2]:
### open Hamiltonian data ###

working_dir = os.getcwd()
parent_dir = os.path.dirname(working_dir) # gets directory where running python file is!

data_dir = os.path.join(parent_dir, 'Molecular_Hamiltonian_data')
hamiltonian_data = os.path.join(data_dir, 'hamiltonians.txt')

In [3]:
with open(hamiltonian_data, 'r') as input_file:
    hamiltonians = ast.literal_eval(input_file.read())

for key in hamiltonians.keys():
    print(f"{key: <25}     n_qubits:  {hamiltonians[key][1]:<5.0f}")

H2-S1_STO-3G_singlet          n_qubits:  18   
C1-O1_STO-3G_singlet          n_qubits:  16   
H1-Cl1_STO-3G_singlet         n_qubits:  16   
H1-Na1_STO-3G_singlet         n_qubits:  16   
H2-Mg1_STO-3G_singlet         n_qubits:  17   
H1-F1_3-21G_singlet           n_qubits:  18   
H1-Li1_3-21G_singlet          n_qubits:  18   
Be1_STO-3G_singlet            n_qubits:  5    
H1-F1_STO-3G_singlet          n_qubits:  8    
H1-Li1_STO-3G_singlet         n_qubits:  8    
Ar1_STO-3G_singlet            n_qubits:  13   
F2_STO-3G_singlet             n_qubits:  15   
H1-O1_STO-3G_singlet          n_qubits:  8    
H2-Be1_STO-3G_singlet         n_qubits:  9    
H2-O1_STO-3G_singlet          n_qubits:  10   
H2_3-21G_singlet              n_qubits:  5    
H2_6-31G_singlet              n_qubits:  5    
H3-N1_STO-3G_singlet          n_qubits:  13   
H4-C1_STO-3G_singlet          n_qubits:  14   
Mg1_STO-3G_singlet            n_qubits:  13   
N2_STO-3G_singlet             n_qubits:  15   
Ne1_STO-3G_si

In [4]:
molecule_key = 'H3_STO-3G_singlet_1+'
# molecule_key= 'Be1_STO-3G_singlet'
transformation, N_qubits, Hamilt_dictionary, _ ,_, _ = hamiltonians[molecule_key]

# 1. Get OpenFermion representation of Hamiltonian

In [5]:
from quchem.Misc_functions.conversion_scripts import Get_Openfermion_Hamiltonian

openFermion_H = Get_Openfermion_Hamiltonian(Hamilt_dictionary)
openFermion_H

-1.7512307459285525 [] +
0.01872992170537467 [X0] +
-0.023568139980123585 [X0 X1] +
0.03597868636603963 [X0 X1 X2] +
-0.023568139980123585 [X0 X1 Z2] +
-0.03597868636603963 [X0 Y1 Y2] +
0.01872992170537467 [X0 Z1] +
0.023568139980123585 [X0 Z1 X2] +
0.01872992170537467 [X0 Z1 Z2] +
0.023568139980123585 [X0 X2] +
0.01872992170537467 [X0 Z2] +
0.03597868636603963 [Y0 X1 Y2] +
-0.023568139980123585 [Y0 Y1] +
0.03597868636603963 [Y0 Y1 X2] +
-0.023568139980123585 [Y0 Y1 Z2] +
0.023568139980123585 [Y0 Z1 Y2] +
0.023568139980123585 [Y0 Y2] +
-0.45436486525596403 [Z0] +
0.02356815233618002 [Z0 X1] +
0.02356815233617983 [Z0 X1 Z2] +
-0.07195737217001562 [Z0 Y1 Y2] +
0.37110605476609804 [Z0 Z1] +
-0.023568152336179825 [Z0 Z1 X2] +
-0.2878474382772282 [Z0 Z1 Z2] +
-0.023568152336180023 [Z0 X2] +
0.37110605476609787 [Z0 Z2] +
0.02356815233618002 [X1] +
0.02356815233617983 [X1 Z2] +
-0.07195737217001562 [Y1 Y2] +
-0.017109477140260287 [Z1] +
-0.023568152336179825 [Z1 X2] +
0.31270210682950855 [Z1 

# 2. Get cliques defined by commutativity 


In [6]:
from quchem.Unitary_Partitioning.Graph import Clique_cover_Hamiltonian

commutativity_flag = 'AC' ## <- defines relationship between sets!!!
Graph_colouring_strategy='largest_first'


anti_commuting_sets = Clique_cover_Hamiltonian(openFermion_H, 
                                                     N_qubits, 
                                                     commutativity_flag, 
                                                     Graph_colouring_strategy)
anti_commuting_sets

{0: [-1.7512307459285525 []],
 1: [-0.017109477140260287 [Z2],
  -0.023568152336179825 [Z1 X2],
  0.03597868636603963 [X0 X1 X2],
  0.023568139980123585 [Y0 Z1 Y2]],
 2: [0.02356815233617983 [X1 Z2],
  -0.017109477140260287 [Z1],
  0.03597868636603963 [Y0 X1 Y2],
  -0.023568139980123585 [Y0 Y1]],
 3: [0.37110605476609787 [Z0 Z2],
  -0.023568152336179825 [Z0 Z1 X2],
  0.01872992170537467 [X0],
  -0.023568139980123585 [Y0 Y1 Z2]],
 4: [0.02356815233617983 [Z0 X1 Z2],
  0.37110605476609804 [Z0 Z1],
  0.01872992170537467 [X0 Z2],
  0.023568139980123585 [X0 Z1 X2]],
 5: [0.023568139980123585 [X0 X2],
  -0.023568139980123585 [X0 X1 Z2],
  0.01872992170537467 [X0 Z1 Z2],
  0.03597868636603963 [Y0 Y1 X2],
  -0.45436486525596403 [Z0]],
 6: [-0.023568139980123585 [X0 X1],
  -0.03597868636603963 [X0 Y1 Y2],
  0.01872992170537467 [X0 Z1],
  0.023568139980123585 [Y0 Y2]],
 7: [-0.023568152336180023 [X2], -0.07195737217001562 [Y1 Y2]],
 8: [0.02356815233618002 [X1], 0.31270210682950855 [Z1 Z2]],
 9:

# 3. custom cirq functions

### 3.1 custom single qubit pauli with phase

In [7]:
from quchem.Unitary_Partitioning.LCU_circuit_functions import singeQ_Pauligate_phase

gate_obj = singeQ_Pauligate_phase('X', -1j)
circuit = cirq.Circuit(gate_obj.on(cirq.LineQubit(1)))
print(circuit)

print('')
print(cirq.Circuit(cirq.decompose(circuit)))

1: ───(-0-1j) X───

1: ───Y───Z───


In [8]:
## CHECKING ALL possibitlies
Pauli_dict = { 'X':np.array([[0.+0.j, 1.+0.j],
                       [1.+0.j, 0.+0.j]]),
                  'Y':np.array([[0.+0.j, 0.-1.j],
                       [0.+1.j, 0.+0.j]]), 
                  'Z':np.array([[ 1.+0.j,  0.+0.j],
                   [ 0.+0.j, -1.+0.j]]), 
                  'I':np.eye(2)}

phase_list = [1, -1, 1j, -1j]

for pstr in Pauli_dict.keys():
    for phase in phase_list:
        gate_obj = singeQ_Pauligate_phase(pstr, phase)
        circuit = cirq.Circuit(gate_obj.on(cirq.LineQubit(1)))
        print(np.allclose(circuit.unitary(), phase*Pauli_dict[pstr]))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


### 3.2 custom PauliWord with phase

In [9]:
from openfermion.ops import QubitOperator
from quchem.Unitary_Partitioning.LCU_circuit_functions import Modified_PauliWord_gate

PauliQubitOp = QubitOperator('Y0 X1 X2 X3',1)
Pn = QubitOperator('Z2',1)
correction_val = -1j
circuit_obj = Modified_PauliWord_gate(PauliQubitOp, correction_val, Pn).on(*list(cirq.LineQubit.range(4)))
circuit=cirq.Circuit(circuit_obj)
print(circuit)

print('')
print('')
print('')
print(cirq.Circuit(cirq.decompose(circuit)))

0: ───Y0───────────
      │
1: ───X1───────────
      │
2: ───(-0-1j)*X2───
      │
3: ───X3───────────



0: ───Y───────

1: ───X───────

2: ───Y───Z───

3: ───X───────


In [10]:
from functools import reduce

## checking
Pword_mat = reduce(np.kron, [Pauli_dict['Y'], Pauli_dict['X'], -1j*Pauli_dict['X'], Pauli_dict['X']])
np.array_equal(circuit.unitary(), Pword_mat)

True

### 3.3 custom PauliWord with phase and controls

In [11]:
OP = QubitOperator('X1 Y2')
correction_val=-1j

qubit_list = cirq.LineQubit.range(3, 5) + cirq.LineQubit.range(3) 
mod_p_word_gate = Modified_PauliWord_gate(OP, correction_val, Pn).controlled(num_controls=2, control_values=[0,1]).on(*qubit_list) 
circuit_with_controls = cirq.Circuit(mod_p_word_gate)
print(circuit_with_controls)

print('')
print('')
print('')
print(cirq.Circuit(cirq.decompose(circuit_with_controls)))

0: ───I────────────
      │
1: ───X1───────────
      │
2: ───(-0-1j)*Y2───
      │
3: ───(0)──────────
      │
4: ───@────────────



1: ───X─────────────────
      │
2: ───┼─────Z─────X─────
      │     │     │
3: ───(0)───(0)───(0)───
      │     │     │
4: ───@─────@─────@─────


In [12]:
## CHECK ## 

zero = np.array([[1],[0]])
one = np.array([[0],[1]])

zero_zero = reduce(np.kron, [zero, zero])
zero_one = reduce(np.kron, [zero, one])
one_zero = reduce(np.kron, [one, zero])
one_one = reduce(np.kron, [one, one])

X=cirq.X._unitary_()
Y=cirq.Y._unitary_()
Z= cirq.Z._unitary_()
I = np.eye(2)

N_sys=3
system_I = np.eye(2**N_sys)

# (I x |00> <00|) + (IX -1jY x |01> <01|) + (I x |10> <10|) + (I x |11> <11|)  
control_op = (np.kron(system_I, np.outer(zero_zero, zero_zero)) + 
             np.kron(reduce(np.kron, [I, X, -1j*Y]), np.outer(zero_one, zero_one)) + 
             np.kron(system_I, np.outer(one_zero, one_zero)) + 
             np.kron(system_I, np.outer(one_one, one_one)))

np.array_equal(control_op, circuit_with_controls.unitary())

True

# 4. Example of R_l operator

In [13]:
key_larg, largest_AC_set = max(anti_commuting_sets.items(), key=lambda x:len(x[1])) # largest nonCon part found by dfs alg
largest_AC_set

[0.023568139980123585 [X0 X2],
 -0.023568139980123585 [X0 X1 Z2],
 0.01872992170537467 [X0 Z1 Z2],
 0.03597868636603963 [Y0 Y1 X2],
 -0.45436486525596403 [Z0]]

In [14]:
## works with:
key=6 #7
key_larg, largest_AC_set = key, anti_commuting_sets[key]
largest_AC_set

[-0.023568139980123585 [X0 X1],
 -0.03597868636603963 [X0 Y1 Y2],
 0.01872992170537467 [X0 Z1],
 0.023568139980123585 [Y0 Y2]]

In [15]:
from quchem.Unitary_Partitioning.Unitary_partitioning_LCU_method import Get_R_op_list

N_index=0
check_reduction = True

R_linear_comb_list, Pn, gamma_l = Get_R_op_list(largest_AC_set,
                                                N_index,
                                                N_qubits,
                                                check_reduction=check_reduction, 
                                                atol=1e-8,
                                                rtol=1e-05)

R_linear_comb_list

[0.5249181768963586 [],
 (-0-0.6527833916355477j) [Z1 Y2],
 (-0-0.33982846654021914j) [Y1],
 0.42761123055588435j [Z0 X1 Y2]]

To perform Unitary Partitioning via a LCU - apply the linear combination of operators in ```R_linear_comb_list```

- in Q circuit form these all must have real amplitudes
- and be l1 normalized

In [16]:
from quchem.Unitary_Partitioning.LCU_circuit_functions import absorb_complex_phases
R_linear_comb_corrected_phase_OP_list, R_linear_comb_correction_values, ancilla_amplitudes, l1_norm = absorb_complex_phases(R_linear_comb_list)

for i, op in enumerate(R_linear_comb_corrected_phase_OP_list):
    print(op, f'correction factor: {R_linear_comb_correction_values[i]}')


0.5249181768963586 [] correction factor: 1
0.6527833916355477 [Z1 Y2] correction factor: (-0-1j)
0.33982846654021914 [Y1] correction factor: (-0-1j)
0.42761123055588435 [Z0 X1 Y2] correction factor: 1j


In [17]:
### checking reduction
from openfermion.ops import QubitOperator
from openfermion.utils import hermitian_conjugated

Hsl = QubitOperator()
for op in largest_AC_set:
    Hsl+=op


### R op without phase absorbed!
R = QubitOperator()
for op in R_linear_comb_list:
    R+=op


print(R*Hsl*hermitian_conjugated(R))
print(Pn * gamma_l)
print('')

### R op WITH phase absorbed!

R_corr = QubitOperator()
for ind, op in enumerate(R_linear_comb_corrected_phase_OP_list):
    phase = R_linear_comb_correction_values[ind]
    R_corr+=op*phase
    
print(R_corr*Hsl*hermitian_conjugated(R_corr))
print(Pn * gamma_l)

(0.05249943127273581+0j) [X0 X1]
0.052499431272735805 [X0 X1]

(0.05249943127273581+0j) [X0 X1]
0.052499431272735805 [X0 X1]


# 5. Checking custom cirq gate functions

### 5.1 Ansatz

In [18]:
qubits = list(cirq.LineQubit.range(N_qubits))

ansatz = cirq.Circuit([cirq.X.on(q) for q in qubits])
ansatz

0: ───X───

1: ───X───

2: ───X───

### 5.2 GUG circuit


note for the different G_methods:
- **only ```'matrix'``` is currently stable**
- ```'cirq_disentangle'``` and ```'IBM'``` may work, but **unstable** (can lead to problems)

In [19]:
from quchem.Unitary_Partitioning.LCU_circuit_functions import Build_GUG_LCU_circuit

N_index=0
N_system_qubits = N_qubits
G_method = 'matrix' # <-- matrix gate method

GUG_circuit, Pn, gamma_l, l1_norm, N_ancilla  = Build_GUG_LCU_circuit(largest_AC_set,
                          N_index, 
                          N_system_qubits,
                          G_method,
                          check_G_circuit=True,
                          allowed_qiskit_gates=['id', 'rz', 'ry', 'rx', 'cx' ,'s', 'h', 'y','z', 'x'], 
                          qiskit_opt_level=0, 
                          check_GUG_circuit=True,
                          check_Rl_reduction_lin_alg=True)
GUG_circuit

0: ───────────────────────────────────────────────────I────────────I────────────1j*Z0───────────────────────────────────────────────────
                                                      │            │            │
1: ───────────────────────────────────────────────────(-0-1j)*Z1───(-0-1j)*Y1───X1──────────────────────────────────────────────────────
                                                      │            │            │
2: ───────────────────────────────────────────────────Y2───────────┼────────────Y2──────────────────────────────────────────────────────
                                                      │            │            │
      ┌                                           ┐   │            │            │       ┌                                           ┐
      │ 0.519+0.j -0.369+0.j -0.381+0.j -0.67 +0.j│   │            │            │       │ 0.519-0.j  0.579+0.j  0.418-0.j  0.469+0.j│
3: ───│ 0.579+0.j  0.815+0.j  0.   +0.j  0.   +0.j│───(0)──────────@────────────@───────│-0.369-0.j  0.815-0.j -0.297-0.j -0.333+0.j│───
      │ 0.418+0.j -0.297+0.j  0.859+0.j  0.   +0.j│   │            │            │       │-0.381-0.j -0.   -0.j  0.859-0.j -0.344-0.j│
      │ 0.469+0.j -0.333+0.j -0.344+0.j  0.742+0.j│   │            │            │       │-0.67 +0.j -0.   -0.j  0.   +0.j  0.742-0.j│
      └                                           ┘   │            │            │       └                                           ┘
      │                                               │            │            │       │
4: ───#2──────────────────────────────────────────────@────────────(0)──────────@───────#2──────────────────────────────────────────────

In [20]:
2**7

128

In [21]:
from quchem.Qcircuit.Circuit_functions_to_create_arb_state import intialization_circuit
start_qubit_ind=0
state_vector = np.array([
    [1/np.sqrt(8)],
    [1/np.sqrt(8)],
    [1/np.sqrt(8)],
    [1/np.sqrt(8)],
    [1/np.sqrt(8)],
    [-1/np.sqrt(8)],
    [1j/np.sqrt(8)],
    [1/np.sqrt(8)]
])
check_circuit=True

t, phase = intialization_circuit(state_vector, 0, check_circuit=check_circuit)
t

0: ───Ry(0.5π)───Rz(0.375π)───@────────────────@───────────────@───────────────@───────────────@────────────────@─────────────────@──────────────────────────────────@──────────────────────────────────@───────────────────────────────────@─────────────────────────────────@──────────────────────────────────@──────────────────────────────────@─────────────────────────────────@─────────────────────────────────@──────────────────────────────────@──────────────────────────────────@─────────────────────────────────@───────────────────────────────
                              │                │               │               │               │                │                 │                                  │                                  │                                   │                                 │                                  │                                  │                                 │                                 │                                  │                                  │                                 │
1: ───────────────────────────X───Ry(-0.25π)───X───Ry(0.25π)───X───Ry(0.25π)───X───Ry(0.25π)───X───Rz(0.125π)───X───Rz(-0.125π)───┼─────────────────@────────────────┼─────────────────@────────────────┼─────────────────@─────────────────┼────────────────@────────────────┼────────────────@─────────────────┼─────────────────@────────────────┼────────────────@────────────────┼────────────────@────────────────┼────────────────@─────────────────┼────────────────@─────────────────┼────────────────@────────────────┼───────────────@───────────────
                                                                                                                                  │                 │                │                 │                │                 │                 │                │                │                │                 │                 │                │                │                │                │                │                │                 │                │                 │                │                │               │
2: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────X───Ry(-0.125π)───X───Ry(0.125π)───X───Ry(-0.125π)───X───Ry(0.125π)───X───Ry(-0.125π)───X───Ry(-0.125π)───X───Ry(0.125π)───X───Ry(0.125π)───X───Ry(0.125π)───X───Ry(-0.125π)───X───Ry(-0.125π)───X───Ry(0.125π)───X───Ry(0.125π)───X───Ry(0.125π)───X───Ry(0.125π)───X───Ry(0.125π)───X───Rz(0.125π)───X───Rz(-0.125π)───X───Rz(0.125π)───X───Rz(-0.125π)───X───Rz(-0.25π)───X───Rz(-0.25π)───X───Rz(0.25π)───X───Rz(0.25π)───

In [22]:
N_index=0
N_system_qubits = N_qubits
G_method = 'cirq_disentangle' # <-- cirq_disentangle method


GUG_circuit, Pn, gamma_l, l1_norm, N_ancilla  = Build_GUG_LCU_circuit(
                                                          largest_AC_set,
                                                          N_index, 
                                                          N_system_qubits,
                                                          G_method,
                                                          check_G_circuit=True,
                                                          allowed_qiskit_gates=['id', 'rz', 'ry', 'rx', 'cx' ,'s', 'h', 'y','z', 'x'], 
                                                          qiskit_opt_level=0, 
                                                          check_GUG_circuit=True,
                                                          check_Rl_reduction_lin_alg=True)
GUG_circuit

0: ─────────────────────────────────────────────────────────────────────────────────────I────────────I────────────1j*Z0────────────────────────────────────────────────────────────────────────────────────────
                                                                                        │            │            │
1: ─────────────────────────────────────────────────────────────────────────────────────(-0-1j)*Z1───(-0-1j)*Y1───X1───────────────────────────────────────────────────────────────────────────────────────────
                                                                                        │            │            │
2: ─────────────────────────────────────────────────────────────────────────────────────Y2───────────┼────────────Y2───────────────────────────────────────────────────────────────────────────────────────────
                                                                                        │            │            │
3: ───Ry(0.432π)───@─────────────────@────────────────@────────────────@────────────────(0)──────────@────────────@─────────────────────@─────────────────@─────────────────@────────────────@───Ry(-0.432π)───
                   │                 │                │                │                │            │            │                     │                 │                 │                │
4: ────────────────X───Ry(-0.268π)───X───Ry(0.268π)───X───Ry(0.267π)───X───Ry(0.267π)───@────────────(0)──────────@───────Ry(-0.267π)───X───Ry(-0.267π)───X───Ry(-0.268π)───X───Ry(0.268π)───X─────────────────

In [23]:
# N_index=0
# N_system_qubits = N_qubits
# G_method = 'IBM' # <-- IBM method


# GUG_circuit, Pn, gamma_l, l1_norm, N_ancilla  = Build_GUG_LCU_circuit(
#                                                           largest_AC_set,
#                                                           N_index, 
#                                                           N_system_qubits,
#                                                           G_method,
#                                                           check_G_circuit=True,
#                                                           allowed_qiskit_gates=['id', 'rz', 'ry', 'rx', 'cx' ,'s', 'h', 'y','z', 'x'], 
#                                                           qiskit_opt_level=0, 
#                                                           check_GUG_circuit=True,
#                                                           check_Rl_reduction_lin_alg=True)
# GUG_circuit

### 5.3 full LCU circuit!

In [24]:
N_ancilla

2

In [25]:
from quchem.Unitary_Partitioning.LCU_circuit_functions import Full_LCU_Rl_Circuit

N_index=0
N_system_qubits = N_qubits
G_method = 'matrix' #'cirq_disentangle'
# anti_commuting_set = anti_commuting_sets[key_larg]
anti_commuting_set = anti_commuting_sets[2]


full_Q_circ, Pn, gamma_l, l1_norm, N_ancilla = Full_LCU_Rl_Circuit(anti_commuting_set,
                                              N_index, 
                                              N_system_qubits,
                                              G_method,
                                              ansatz,
                                              check_G_circuit=True,
                                              allowed_qiskit_gates=['id', 'rz', 'ry', 'rx', 'cx' ,'s', 'h', 'y','z', 'x'], 
                                              qiskit_opt_level=0, 
                                              check_GUG_circuit=True,
                                              check_Rl_reduction_lin_alg=True)

full_Q_circ

0: ───X───────────────────────────────────────────────I───────Y0───────────Y0───────────────────────────────────────────────────────────────
                                                      │       │            │
1: ───X───────────────────────────────────────────────1j*Y1───I────────────(-0-1j)*Z1───Ry(-0.5π)───────────────────────────────────────M───
                                                      │       │            │                                                            │
2: ───X───────────────────────────────────────────────Z2──────(-0-1j)*X2───Z2───────────────────────────────────────────────────────────M───
                                                      │       │            │                                                            │
      ┌                                           ┐   │       │            │            ┌                                           ┐   │
      │ 0.704+0.j -0.251+0.j -0.451+0.j -0.488+0.j│   │       │            │            │ 0.704+0.j  0.335-0.j  0.486-0.j  0.394-0.j│   │
3: ───│ 0.335+0.j  0.942+0.j  0.   +0.j  0.   +0.j│───(0)─────@────────────@────────────│-0.251-0.j  0.942+0.j -0.173-0.j -0.14 +0.j│───M───
      │ 0.486+0.j -0.173+0.j  0.857+0.j  0.   +0.j│   │       │            │            │-0.451-0.j  0.   +0.j  0.857+0.j -0.252-0.j│   │
      │ 0.394+0.j -0.14 +0.j -0.252+0.j  0.873+0.j│   │       │            │            │-0.488-0.j  0.   +0.j  0.   +0.j  0.873-0.j│   │
      └                                           ┘   │       │            │            └                                           ┘   │
      │                                               │       │            │            │                                               │
4: ───#2──────────────────────────────────────────────@───────(0)──────────@────────────#2──────────────────────────────────────────────M───

### 5.4 Checking GUGdag = Rl

In [26]:
from quchem.Unitary_Partitioning.LCU_circuit_functions import Build_GUG_LCU_circuit

N_index=0
N_system_qubits = N_qubits
G_method = 'matrix' # <-- matrix gate method


GUG_circuit, Pn, gamma_l, l1_norm, N_ancilla  = Build_GUG_LCU_circuit(largest_AC_set,
                          N_index, 
                          N_system_qubits,
                          G_method,
                          check_G_circuit=True,
                          allowed_qiskit_gates=['id', 'rz', 'ry', 'rx', 'cx' ,'s', 'h', 'y','z', 'x'], 
                          qiskit_opt_level=0, 
                          check_GUG_circuit=True,
                          check_Rl_reduction_lin_alg=True)
GUG_circuit

0: ───────────────────────────────────────────────────I────────────I────────────1j*Z0───────────────────────────────────────────────────
                                                      │            │            │
1: ───────────────────────────────────────────────────(-0-1j)*Z1───(-0-1j)*Y1───X1──────────────────────────────────────────────────────
                                                      │            │            │
2: ───────────────────────────────────────────────────Y2───────────┼────────────Y2──────────────────────────────────────────────────────
                                                      │            │            │
      ┌                                           ┐   │            │            │       ┌                                           ┐
      │ 0.519+0.j -0.369+0.j -0.381+0.j -0.67 +0.j│   │            │            │       │ 0.519-0.j  0.579+0.j  0.418-0.j  0.469+0.j│
3: ───│ 0.579+0.j  0.815+0.j  0.   +0.j  0.   +0.j│───(0)──────────@────────────@───────│-0.369-0.j  0.815-0.j -0.297-0.j -0.333+0.j│───
      │ 0.418+0.j -0.297+0.j  0.859+0.j  0.   +0.j│   │            │            │       │-0.381-0.j -0.   -0.j  0.859-0.j -0.344-0.j│
      │ 0.469+0.j -0.333+0.j -0.344+0.j  0.742+0.j│   │            │            │       │-0.67 +0.j -0.   -0.j  0.   +0.j  0.742-0.j│
      └                                           ┘   │            │            │       └                                           ┘
      │                                               │            │            │       │
4: ───#2──────────────────────────────────────────────@────────────(0)──────────@───────#2──────────────────────────────────────────────

In [27]:
G_U_G_dag =  GUG_circuit.unitary()

In [28]:
from functools import reduce

# Use POVM to force all zero ancilla measurment (Identity on system reg)
I_sys = np.eye(2**N_system_qubits)
ancilla_0_state = reduce(np.kron, [np.array([[1],[0]]) for _ in range(N_ancilla)])
ancilla_0_projector = np.outer(ancilla_0_state, ancilla_0_state)

POVM_0_ancilla = np.kron(I_sys, ancilla_0_projector) # forces all zero measurement on ancilla!
projected_GUG = POVM_0_ancilla.dot(G_U_G_dag)

trace_GUG = projected_GUG.reshape([2 ** N_system_qubits, 2 ** N_ancilla,
                                                2 ** N_system_qubits, 2 ** N_ancilla])
reduced_MAT = np.einsum('jiki->jk', trace_GUG)

In [29]:
from openfermion.ops import QubitOperator
from functools import reduce
# Get Rl operator

Rl = reduce(lambda Op1, Op2: Op1+Op2, R_linear_comb_list)
Rl

0.5249181768963586 [] +
0.42761123055588435j [Z0 X1 Y2] +
-0.33982846654021914j [Y1] +
-0.6527833916355477j [Z1 Y2]

In [30]:
from openfermion import qubit_operator_sparse
Rl_mat = qubit_operator_sparse(Rl, n_qubits = N_system_qubits)

In [31]:
# check if POVM of GUG is same as Rl
print(np.allclose(Rl_mat.todense(), reduced_MAT*l1_norm))

True


# 6. Linear Algebra circuit experiment

### 6.1 define Ansatz

In [32]:
from openfermion.linalg import qubit_operator_sparse
from scipy.linalg import eigh

H_matrix = qubit_operator_sparse(openFermion_H)
eig_values, eig_vectors = eigh(H_matrix.todense()) # NOT sparse!

idx = eig_values.argsort()  
eigenValues = eig_values[idx]
eigenVectors = eig_vectors[:,idx]

ground_state = np.around(eigenVectors[:,0].real, 10)

In [33]:
ground_state.conj().T @ H_matrix.todense() @ ground_state

matrix([[-2.91601849+0.j]])

In [34]:
from quchem.Qcircuit.Circuit_functions_to_create_arb_state import prepare_arb_state_cirq_matrix_gate
ansatz_circuit = prepare_arb_state_cirq_matrix_gate(ground_state,
                             start_qubit_ind=0)
ansatz_circuit

┌                                                                 ┐
      │-0.094+0.j -0.   +0.j  0.   +0.j  0.701+0.j -0.707+0.j  0.   +0.j│
      │   0.   +0.j  0.   +0.j                                          │
      │-0.   +0.j  1.   +0.j -0.   +0.j -0.   +0.j  0.   +0.j  0.   +0.j│
      │   0.   +0.j  0.   +0.j                                          │
      │ 0.   +0.j  0.   +0.j  1.   +0.j  0.   +0.j -0.   +0.j  0.   +0.j│
      │   0.   +0.j  0.   +0.j                                          │
      │ 0.991+0.j  0.   +0.j -0.   +0.j  0.133+0.j  0.   +0.j  0.   +0.j│
0: ───│   0.   +0.j  0.   +0.j                                          │───
      │-0.094+0.j -0.   +0.j  0.   +0.j  0.701+0.j  0.707+0.j  0.   +0.j│
      │   0.   +0.j  0.   +0.j                                          │
      │ 0.   +0.j  0.   +0.j  0.   +0.j  0.   +0.j  0.   +0.j  1.   +0.j│
      │   0.   +0.j  0.   +0.j                                          │
      │ 0.   +0.j  0.   +0.j  0.   +0.j  0.   +0.j  0.   +0.j  0.   +0.j│
      │   1.   +0.j  0.   +0.j                                          │
      │ 0.   +0.j  0.   +0.j  0.   +0.j  0.   +0.j  0.   +0.j  0.   +0.j│
      │   0.   +0.j  1.   +0.j                                          │
      └                                                                 ┘
      │
1: ───#2────────────────────────────────────────────────────────────────────
      │
2: ───#3────────────────────────────────────────────────────────────────────

# 6.2 Run Unitary Partitioning LCU experiment

(gets all matrices from quantum circuits)

In [35]:
from quchem.Unitary_Partitioning.LCU_circuit_functions import LCU_VQE_Experiment_UP_circuit_lin_alg

G_method = 'matrix'
exp = LCU_VQE_Experiment_UP_circuit_lin_alg(anti_commuting_sets, 
                                            ansatz_circuit, 
                                            N_system_qubits,
                                            G_method,
                                            N_indices_dict=None,
                                            check_G_circuit=True,
                                              allowed_qiskit_gates=['id', 'rz', 'ry', 'rx', 'cx' ,'s', 'h', 'y','z', 'x'], 
                                              qiskit_opt_level=0, 
                                              check_GUG_circuit=True,
                                              check_Rl_reduction_lin_alg=True)

exp.Calc_Energy()

-2.9160184902684527

In [36]:
G_method='cirq_disentangle'

exp = LCU_VQE_Experiment_UP_circuit_lin_alg(anti_commuting_sets, 
                                            ansatz_circuit, 
                                            N_system_qubits,
                                            G_method,
                                            N_indices_dict=None,
                                            check_G_circuit=True,
                                              allowed_qiskit_gates=['id', 'rz', 'ry', 'rx', 'cx' ,'s', 'h', 'y','z', 'x'], 
                                              qiskit_opt_level=0, 
                                              check_GUG_circuit=True,
                                              check_Rl_reduction_lin_alg=True)

exp.Calc_Energy()

-2.916018490268453